In [7]:
#!/usr/bin/env python3

from http.client  import HTTPConnection
from urllib.parse import urlencode
from arcgis.mapping import create_symbol
import pandas as pd
import numpy as np
import json
from datetime import datetime, date, time

#------------------------------------------------------
# Runs SPARQL query at SPARQL endpoint and
# return results as a Python 'dict' (in the SPARQL1.1 results format)
# (for SPARQL1.1 results format refer: https://www.w3.org/TR/sparql11-results-json)
#
#       sparql_endpoint: 'host:port' ex: '192.168.0.64:7070', 'data.nobelprize.org'
#       sparql_query: ex: 'select (count(*) as ?c) {?s?p?o}'
#       fmt - optional argument, if specified, returns results in a raw string format
#              possiblea values ('csv','json','xml'), any other format will be treated as 'json'
#------------------------------------------------------
#
def run_query(sparql_endpoint,sparql_query,fmt=None):
   # create HTTP connection to SPARQL endpoint
   conn = HTTPConnection(sparql_endpoint,timeout=100) #may throw HTTPConnection exception
   # urlencode query for sending
   docbody = urlencode({'query':sparql_query})
   # request result in json
   hdrs = {'Accept': 'application/sparql-results+json',
           'Content-type': 'application/x-www-form-urlencoded'}
   raw = False
   if fmt is not None:
      raw = True
      if fmt in ('xml','XML'):
         hdrs['Accept'] = 'application/sparql-results+xml'
      elif fmt in ('csv','CSV'):
         hdrs['Accept'] = 'text/csv, application/sparql-results+csv'

   # send post request
   conn.request('POST','/sparql',docbody,hdrs) #may throw exception

   # read response
   resp = conn.getresponse()
   if 200 != resp.status:
      errmsg = resp.read()
      conn.close()
      raise Exception('Query Error',errmsg)  # query processing errors - syntax errors, etc.

   # content-type header, and actual response data
   ctype = resp.getheader('content-type','text/html').lower()
   result = resp.read().lstrip()
   conn.close()

   # check response content-type header
   if raw or ctype.find('json') < 0:
      return result      # not a SELECT?

   # convert result in JSON string into python dict
   return json.loads(result)


#------------------------------------------------------
# Returns pandas DataFrame from the results of running a sparql_query at sparql_endpoint
#       sparql_endpoint: 'host:port' ex: '192.168.0.64:7070', 'data.nobelprize.org'
#       sparql_query: ex: 'select (count(*) as ?c) {?s?p?o}'
#------------------------------------------------------
#
def create_dataframe(sparql_endpoint,sparql_query):
   # run query
   result = run_query(sparql_endpoint,sparql_query)  # may throw exception
   # result is in SPARQL results format refer: https://www.w3.org/TR/sparql11-results-json/
   cols = result.get('head',{}).get('vars',[])
   rows = result.get('results',{}).get('bindings',[])

   # extract types and columnar data for rows
   coltype = {}
   nptype = {}
   coldata = {}
   for col in cols:
      coltype[col] = None
      coldata[col] = []
      nptype[col] = None

   # for all rows, save (columnar) data in coldata[] for each col
   for row in rows:
      for col in cols:
         cell = row.get(col,None)
         if cell is None:  # unbound value
            val = None
            if coltype[col] in ('byte','short','int','integer','float','double','decimal'):
               val = np.nan #missing numeric values as NaN
            coldata[col].append(val)
            continue
         # compute type and datum
         pdval = cell.get('value','')
         vtype = cell.get('type','')
         langtag = cell.get('xml:lang','')
         typeuri = cell.get('datatype','')
         pdtype = 'object'
         if vtype == 'uri':
            pdval = '<'+pdval+'>'
         elif langtag != '':
            pdval = '"'+pdval+'"@'+langtag
            coltype[col] = 'object'
         elif typeuri != '':
            #vtype in ('typed-literal')
            typeuri = typeuri.replace('http://www.w3.org/2001/XMLSchema#','')
            coltype[col] = typeuri if (coltype[col] is None or coltype[col] == typeuri) else 'object'
            pdtype,pdval = typed_value(typeuri,pdval)
         nptype[col] = pdtype if (coltype[col] != 'object') else 'object'
         coldata[col].append(pdval) # columnar data
   # instantiate DataFrame
   npdata = {}
   for col in cols:
      npdata[col] = np.array(coldata[col],dtype=np.dtype(nptype[col]))
   return pd.DataFrame(columns=cols,data=npdata)

# util: convert literal val into typed-value based on the typeuri
def typed_value(typeuri,val):
   # {"duration", ColTypeDuration},
   if typeuri in ('boolean'):
      return np.bool, 'true' == val
   elif typeuri in ('byte'):
      return np.byte, np.int8(val)
   elif typeuri in ('short'):
      return np.short, np.short(val)
   elif typeuri in ('integer','int','nonNegativeInteger'):
      return np.intc, np.int(val)
   elif typeuri in ('long'):
      return np.int_, np.int_(val)
   elif typeuri in ('float'):
      return np.single, np.float32(val)
   elif typeuri in ('double', 'decimal'):
      return np.double, np.float64(val)
   elif typeuri in ('dateTime'):
      return np.datetime64, datetime.fromisoformat(val)
   elif typeuri in ('date'):
      return pd.date, date.fromisoformat(val)
   elif typeuri in ('time'):
      return pd.time, time.fromisoformat(val)
   return 'object', val


In [8]:
# query
query = '''PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix : <https://ontologies.semanticarts.com/raw_data#>
prefix fl: <https://ontologies.semanticarts.com/flights/>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix skos:    <http://www.w3.org/2004/02/skos/core#>
CONSTRUCT { 
  	?orig fl:hasRouteTo ?dest .
  	?orig rdfs:label ?origCode . 
  	?dest rdfs:label ?destCode . 
  	fl:hasRouteto rdfs:label "hasRouteTo" . 
}  
WHERE { GRAPH <airline_flight_network> {{
SELECT 
?orig ?dest ?origCode ?destCode ?miles 
    WHERE { 
        ?orig fl:terminalCode ?origCode .
        ?orig fl:hasRouteTo ?dest .
        ?dest fl:terminalCode ?destCode .
        << ?orig fl:hasRouteTo ?dest >> fl:distanceMiles ?miles .
        FILTER (?miles < 400)
        FILTER (?origCode = 'BOS')
  }
      }}}
'''

originAirport = 'BOS'
flights = run_query('localhost:7070',query)

x = flights.decode("utf-8")

a = x.split(".\n")
a = a[1: len(a)-1]
a

['<https://data.semanticarts.com/flights/_Airport_BOS> <https://ontologies.semanticarts.com/flights/hasRouteTo> <https://data.semanticarts.com/flights/_Airport_JFK> ',
 '<https://data.semanticarts.com/flights/_Airport_BOS> <http://www.w3.org/2000/01/rdf-schema#label> "BOS" ',
 '<https://data.semanticarts.com/flights/_Airport_JFK> <http://www.w3.org/2000/01/rdf-schema#label> "JFK" ',
 '<https://ontologies.semanticarts.com/flights/hasRouteto> <http://www.w3.org/2000/01/rdf-schema#label> "hasRouteTo" ',
 '<https://data.semanticarts.com/flights/_Airport_BOS> <https://ontologies.semanticarts.com/flights/hasRouteTo> <https://data.semanticarts.com/flights/_Airport_BWI> ',
 '<https://data.semanticarts.com/flights/_Airport_BOS> <http://www.w3.org/2000/01/rdf-schema#label> "BOS" ',
 '<https://data.semanticarts.com/flights/_Airport_BWI> <http://www.w3.org/2000/01/rdf-schema#label> "BWI" ',
 '<https://ontologies.semanticarts.com/flights/hasRouteto> <http://www.w3.org/2000/01/rdf-schema#label> "has

In [15]:
s = set()
for ele in a:
    s.add(ele)

nodes = []
for ele in s:
    arr = ele.split(' ')
    if arr[2][0] == '\"' and arr[2][len(arr[2])-1] == '\"':  #change line
        if len(arr[2]) == 5:
            nodes.append({'data': {'id':arr[2][1:len(arr[2])-1], 
                                   'label':arr[2][1:len(arr[2])-1]
                                  }
                         })
        else:
            edgeName = arr[2][1:len(arr[2])-1]
        
edges = []     
for ele in s:
    arr = ele.split(' ')
    if edgeName in arr[1]:
        edges.append({'data': {'source':arr[0][len(arr[0])-4:len(arr[0])-1],
                               'target': arr[2][len(arr[2])-4:len(arr[2])-1]
                              },
                      'classes' : 'origin'
#                       ,
#                       'style': {'label':nodeName}
                     })

# create sets of nodes of S's and O's with <> | When label is found  
edges

[{'data': {'source': 'BOS', 'target': 'JFK'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'LGA'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'MVY'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'ACK'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'ACY'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'DCA'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'EWR'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'BWI'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'PHL'}, 'classes': 'origin'},
 {'data': {'source': 'BOS', 'target': 'BUF'}, 'classes': 'origin'}]

In [17]:
import dash
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

app = JupyterDash(__name__)

elements = nodes + edges


app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='grid',
        clearable=False,
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['grid', 'random', 'circle', 'cose', 'concentric']
        ]
    ),
    cyto.Cytoscape(
        id='cytoscape-update-layout',
        layout={'name': 'concentric'},
        style={'width': '100%', 'height': '450px'},
        elements=elements,
        stylesheet = [
            {
            'selector': 'node',
            'style': {
                'content': 'data(label)',
                'background-color': 'blue',
                }
            },
            {
            'selector': 'edge',
            'style': {
                'content': edgeName,
                'curve-style': 'bezier',
                'target-arrow-color': 'red',
                'target-arrow-shape': 'triangle',
                'line-color': 'red'
                }
            },
        ]
    )
])


@app.callback(Output('cytoscape-update-layout', 'layout'),
              [Input('dropdown-update-layout', 'value')])
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }


port = '8090'
host = '127.0.0.1'

app.run_server(mode='inline',port = port, host = host)

print('App running on http://'+ host +':' + port +'/')

App running on http://127.0.0.1:8090/


In [14]:
elements

[{'data': {'id': 'EWR', 'label': 'EWR'}},
 {'data': {'id': 'ACY', 'label': 'ACY'}},
 {'data': {'id': 'BUF', 'label': 'BUF'}},
 {'data': {'id': 'DCA', 'label': 'DCA'}},
 {'data': {'id': 'PHL', 'label': 'PHL'}},
 {'data': {'id': 'MVY', 'label': 'MVY'}},
 {'data': {'id': 'ACK', 'label': 'ACK'}},
 {'data': {'id': 'JFK', 'label': 'JFK'}},
 {'data': {'id': 'BWI', 'label': 'BWI'}},
 {'data': {'id': 'BOS', 'label': 'BOS'}},
 {'data': {'id': 'LGA', 'label': 'LGA'}},
 {'data': {'source': 'BOS', 'target': 'JFK'}, 'classes': 'origin-BOS'},
 {'data': {'source': 'BOS', 'target': 'LGA'}, 'classes': 'origin-BOS'},
 {'data': {'source': 'BOS', 'target': 'MVY'}, 'classes': 'origin-BOS'},
 {'data': {'source': 'BOS', 'target': 'ACK'}, 'classes': 'origin-BOS'},
 {'data': {'source': 'BOS', 'target': 'ACY'}, 'classes': 'origin-BOS'},
 {'data': {'source': 'BOS', 'target': 'DCA'}, 'classes': 'origin-BOS'},
 {'data': {'source': 'BOS', 'target': 'EWR'}, 'classes': 'origin-BOS'},
 {'data': {'source': 'BOS', 'targe